In [1]:
import cbayes.sample
import cbayes.distributions
import cbayes.solve
import numpy as np
import ipywidgets as wd
import matplotlib.pyplot as plt
import scipy.stats as sstats
import scipy.spatial as spat

The following linear map $Q_s: \mathbb{R}^2 \to \mathbb{R}^2$ is defined to have skewness $s$ at all $\lambda \in \Lambda$.  

$$
Q_s(\lambda) = \lbrace \, \lambda_1, \; \lambda_1 \sqrt{s^2 - 1} + \lambda_2 \, \rbrace
$$

### Useful Identities
Let $\lambda$ denote an arbitrary Gaussian random variable with mean $\mu_\lambda$ and covariance $\Sigma_\lambda$,
$$
\lambda \sim N\left(\mu_\lambda, \Sigma_\lambda\right).
$$
Then, for a matrix $A$, 
$$
A\lambda \sim N\left(A\mu_\lambda,\, A\Sigma_\lambda A^T\right)
$$
Let $\eta = A\lambda + e$, where $e\sim N(0,\Sigma_e)$,   
then the posterior $p(\lambda | \eta)$ is given by

$$
p(\lambda | \eta=\bar{\eta}) = N\left(\hat{\mu}, \hat{\Sigma}\right),
$$
where 
$$
\hat{\mu} = \mu_\lambda + \Sigma_\lambda A^T\left(A\Sigma_\lambda A^T + \Sigma_e\right)^{-1}\left(\bar{\eta} - A\mu_\lambda\right)
$$
and
$$
\hat{\Sigma} = \Sigma_\lambda - \Sigma_\lambda A^T\left(A\Sigma_\lambda A^T + \Sigma_e\right)^{-1}A\Sigma_\lambda
$$

[These notes](https://cs.nyu.edu/~roweis/notes/gaussid.pdf) by Sam Roweis also provide some useful identities.


### Define a function that generates an arbitrarily ill-condidtioned 2-2 map


In [2]:
def make_model(skew):
    # this function makes a linear map whos first component is the x-unit vector
    # and each subsequent component is a norm-1 vector satisfying the property
    # that the 2-2 map made from it and the aforementioned unit vector is a map
    # with skewness in skew_range, which is a list of desired skewnesses   
    # TODO currently this map only works for 2-D input space     
    
    def my_model(parameter_samples):
        Q_map = skewmat(skew)
        QoI_samples = np.dot(parameter_samples, np.transpose(Q_map))
#         QoI_samples = Q_map@parameter_samples.T
        return QoI_samples
    return my_model

def skewmat(skew):
    Q_map = [ [1.0, 0.0] ] # all map components have the same norm, rect_size to have measures of events equal btwn spaces.
    Q_map.append( [np.sqrt(skew**2 - 1), 1] ) # taken with the first component, this leads to a 2-2 map with skewsness 's'
    Q_map = np.array( Q_map )
    return Q_map

def gauss_sol(prior_mean, prior_std, data_std, A, data):
    if type(prior_mean) is int:
        prior_mean = [prior_mean, prior_mean]
    if type(prior_mean) is float:
        prior_mean = [prior_mean, prior_mean]
    if type(prior_mean) is list:
        prior_mean = np.array(prior_mean).reshape(-1,1)
    if type(prior_std) is list:
        prior_std = np.array(prior_std).reshape(-1,1)
    if type(data_std) is list:
        data_std = np.array(data_std).reshape(-1,1)
    prior_cov = prior_std*prior_std*np.eye(2) 
    data_cov = data_std*data_std*np.eye(2) 
    
    ASA = A@prior_cov@A.T
    
    precision = np.linalg.inv(ASA + data_cov)
    kahlman_update = (prior_cov@A.T@precision)
    post_mean = prior_mean + kahlman_update@(data - A@prior_mean)
    post_cov = prior_cov - kahlman_update@A@prior_cov
    
    return prior_mean, prior_cov, post_mean, post_cov

Example of Analytical Solution

In [3]:
# A = skewmat(1.01)
# # print(A)
# data_std = 0.25
# prior_std = 1
# prior_mean = 0
# lam_true = np.array([0.0, 0.0])
# obs_data = A@lam_true.T + data_std*np.random.randn(2)

# prior_mean, prior_cov, post_mean, post_cov = gauss_sol(prior_mean, prior_std, data_std, A, obs_data.reshape(-1,1) )
# print(post_mean.T,'\n')
# print(post_cov)


#### Visualize the contours of this vector-valued map.

### Generate samples and map them

In [67]:
def compare(num_samples=5000, skew=1, prior_x=0.0, prior_y=0.0, prior_std = 1.0, data_std = 0.25, color_map = 'jet', num_levels = 40, seed=12):
    model = make_model(skew)
    lam_true = np.array([0.0, 0.0])
    obs_data = model(lam_true)
    np.random.seed(seed)
    obs_data_noisy = obs_data + data_std*np.random.randn(2)
    mse_fun = cbayes.sample.MSE_generator(model, obs_data_noisy, data_std)
    
    # ANALYTICAL SOLUTION
    prior_mean = np.array([prior_x, prior_y])
    A = skewmat(skew)
    prior_mean, prior_cov, post_mean, post_cov = gauss_sol(prior_mean, prior_std, data_std, A, obs_data_noisy)

    s_input_set = cbayes.sample.sample_set(size=(num_samples, 2))
    s_input_set.set_dist(dim=0, distribution='normal', kwds={'loc': prior_mean[0], 'scale': prior_std})
    s_input_set.set_dist(dim=1, distribution='normal', kwds={'loc': prior_mean[1], 'scale': prior_std})
    input_samples = s_input_set.generate_samples(seed=seed)
    
    output_samples_vector_valued = model(input_samples)
    output_samples_scalar_valued = mse_fun(input_samples)
    
    
    
    # VECTOR PROBLEM
    s_output_set_vector_valued = cbayes.sample.sample_set(size=(num_samples, 2))
    s_output_set_vector_valued.samples = output_samples_vector_valued
    p_set_vector = cbayes.sample.problem_set(s_input_set, s_output_set_vector_valued)
    # Set observed 
    p_set_vector.set_observed_dist(dim=0, dist='normal', 
                                   kwds={'loc': obs_data_noisy[0], 'scale': data_std})
    p_set_vector.set_observed_dist(dim=1, dist='normal', 
                                   kwds={'loc': obs_data_noisy[1], 'scale': data_std})
    p_set_vector.model = model
    
    p_set_vector.compute_pushforward_dist()
    p_set_vector.set_ratio()
    
    cbayes.solve.problem(p_set_vector)
    accepted_inputs_vector = p_set_vector.input.samples[p_set_vector.accept_inds,:]
    print('num accepted for vector-valued:', len(accepted_inputs_vector), 
          'mean: %2.4f, %2.4f'%(np.mean(accepted_inputs_vector[:,0]), np.mean(accepted_inputs_vector[:,1])), 
          'sd: %2.4f, %2.4f'%(np.std(accepted_inputs_vector[:,0]), np.std(accepted_inputs_vector[:,1])) )

    # SCALAR PROBLEM
    s_output_set_scalar_valued = cbayes.sample.sample_set(size=(num_samples, 1))
    s_output_set_scalar_valued.samples = output_samples_scalar_valued
    p_set_scalar = cbayes.sample.problem_set(s_input_set, s_output_set_scalar_valued)
    # Set observed 
    num_observations = 2 # NOT TO BE CHANGED
    p_set_scalar.set_observed_dist('gamma', {'a':num_observations/2.0, 'scale':2.0/num_observations}, dim=0)
    p_set_scalar.model = mse_fun
    
    p_set_scalar.compute_pushforward_dist()
    p_set_scalar.set_ratio()
    
    cbayes.solve.problem(p_set_scalar)
    accepted_inputs_scalar = p_set_scalar.input.samples[p_set_scalar.accept_inds,:]
    print('num accepted for scalar-valued:', len(accepted_inputs_scalar), 
          'mean: %2.4f, %2.4f'%(np.mean(accepted_inputs_scalar[:,0]), np.mean(accepted_inputs_scalar[:,1])), 
          'sd: %2.4f, %2.4f'%(np.std(accepted_inputs_scalar[:,0]), np.std(accepted_inputs_scalar[:,1])) )
    
    # PLOTTING
    fig, axs = plt.subplots(ncols=3, nrows=3, figsize=(15,15))
    
    x = input_samples[:,0] 
    y = input_samples[:,1]
    xs = accepted_inputs_scalar[:,0]
    ys = accepted_inputs_scalar[:,1]

    xv = accepted_inputs_vector[:,0]
    yv = accepted_inputs_vector[:,1]
    
    
    # CONTOURS FOR FORWARD MAP
    axs[0,0].tricontour(x, y, output_samples_vector_valued[:,0], num_levels, cmap=color_map)
    axs[0,0].tricontour(x, y, output_samples_vector_valued[:,1], num_levels, cmap=color_map)
    axs[0,1].tricontour(x, y, output_samples_scalar_valued, num_levels, cmap=color_map)
    
    # MESH PLOT
    vmin, vmax = 0, None
    vpost = p_set_vector.ratio*s_input_set.dist.pdf(input_samples)
    spost = p_set_scalar.ratio*s_input_set.dist.pdf(input_samples)
    axs[1,0].tricontourf(x, y, vpost, int(num_levels/2), cmap=color_map,
                          vmin=vmin, vmax=vmax)
    axs[1,1].tricontourf(x, y, spost, int(num_levels/2), cmap=color_map,
                          vmin=vmin, vmax=vmax)
    
    # CONTOURS OF TRUE POSTERIOR
    post_pdf = sstats.multivariate_normal.pdf(input_samples, mean=post_mean, cov=post_cov, allow_singular=True)
    axs[1,0].tricontour(x, y, post_pdf, int(num_levels/2), cmap='Greys', 
                        vmin=vmin, vmax=vmax, alpha=0.25)
    axs[1,1].tricontour(x, y, post_pdf, int(num_levels/2), cmap='Greys', 
                        vmin=vmin, vmax=vmax, alpha=0.25)
    prior_pdf = sstats.multivariate_normal.pdf(input_samples, mean=prior_mean, cov=prior_cov, allow_singular=True)
    
    # PRIOR CONTOURS
    axs[0,0].tricontour(x, y, prior_pdf, int(num_levels/2), cmap='Greys', 
                        vmin=vmin, vmax=vmax, alpha=0.25)
    axs[0,1].tricontour(x, y, prior_pdf, int(num_levels/2), cmap='Greys', 
                        vmin=vmin, vmax=vmax, alpha=0.25)
    
    # ERRORS 
    vdist = np.linalg.norm(vpost-post_pdf,1)
    sdist = np.linalg.norm(spost-post_pdf,1)
    bdist = np.linalg.norm(spost-vpost,1)
    print('L-1 error: %2.2e vector | %2.2e scalar | %2.2e each'%(vdist, sdist, bdist))

    
    bound_val = 1.0
    axs[0,0].axis([-bound_val, bound_val, -bound_val, bound_val])
    axs[0,1].axis([-bound_val, bound_val, -bound_val, bound_val])
    axs[1,0].axis([-bound_val, bound_val, -bound_val, bound_val])
    axs[1,1].axis([-bound_val, bound_val, -bound_val, bound_val])
    axs[0,0].scatter(lam_true[0], lam_true[1], 100, 'k') # plot true value to compare
    axs[0,1].scatter(lam_true[0], lam_true[1], 100, 'k')

    axs[0,0].scatter(obs_data_noisy[0], obs_data_noisy[1], 100, 'r') # plot true value to compare
    axs[0,1].scatter(obs_data_noisy[0], obs_data_noisy[1], 100, 'r')
    
    # SHOW TRUTH AS WHITE DOT WITH BLACK BORDER
    axs[1,0].scatter(lam_true[0], lam_true[1], 100, 'k')
    axs[1,1].scatter(lam_true[0], lam_true[1], 100, 'k')
    axs[1,0].scatter(lam_true[0], lam_true[1], 60, 'w')
    axs[1,1].scatter(lam_true[0], lam_true[1], 60, 'w')
    
    # SCATTERPLOT ACCEPTED SAMPLES
#     axs[1,0].scatter(xv, yv, color='w', alpha=0.5)
#     axs[1,1].scatter(xs, ys, color='w', alpha=0.5)
    
    # PLOT PUSH-FORWARD AND OBSERVED FOR SCALAR MAP
    num_plot_pts = 100
    xx = np.linspace(0, 20, num_plot_pts)
    X, Y = np.meshgrid(xx,xx)
    D = np.vstack([X.ravel(), Y.ravel()]).T
    axs[2,2].plot(xx, p_set_scalar.pushforward_dist.pdf(xx))
    axs[2,2].plot(xx, p_set_scalar.observed_dist.pdf(xx))
    
    # PLOT DATA FOR VECTORS
    xx = np.linspace(-2.0, 2.0, num_plot_pts)
    axs[0,2].plot(xx, sstats.distributions.norm.pdf(xx,loc=obs_data_noisy[0],scale=data_std), 
                  color='blue', ls='-', label='$q_1$')
    axs[1,2].plot(xx, sstats.distributions.norm.pdf(xx,loc=obs_data_noisy[1], scale=data_std), 
                  color='orange', ls='--', label='$q_2$')
    axs[0,2].vlines(obs_data_noisy[0], 0, 1, 'r', lw=1)
    axs[0,2].vlines(obs_data[0], 0, 0.5, 'k', lw=1)
    axs[1,2].vlines(obs_data[1], 0, 0.5, 'k', lw=1)
    axs[1,2].vlines(obs_data_noisy[1], 0, 1, 'r', lw=1)
    
    
    xxx = np.linspace(-bound_val, bound_val,num_plot_pts)
    XX, YY = np.meshgrid(xxx,xxx)
    lam = np.vstack([XX.ravel(), YY.ravel()]).T

    normalize_marginals = False
    # PLOTTING MARGINALS (both on same axis)
    zzs = p_set_scalar.evaluate_posterior(lam)
    zzs = zzs.reshape(num_plot_pts,num_plot_pts)
    if normalize_marginals:
        zzs = zzs/np.sum(zzs)
    marg_sx = np.sum(zzs, axis=0)
    marg_sy = np.sum(zzs, axis=1)
    line0, = axs[2,1].plot(xxx, marg_sx,  color='blue', ls='-', lw=2)
    line1, = axs[2,1].plot(xxx, marg_sy, color='orange', ls='--', lw=2)
    plt.legend([line0, line1], ['$\\lambda_1$', '$\\lambda_2$' ])
    
    zzv = p_set_vector.evaluate_posterior(lam)
    zzv = zzv.reshape(num_plot_pts,num_plot_pts)
    if normalize_marginals:
        zzv = zzv/np.sum(zzv)
    marg_vx = np.sum(zzv, axis=0)
    marg_vy = np.sum(zzv, axis=1)
    line3, = axs[2,0].plot(xxx, marg_vx,  color='blue', ls='-',  lw=2)
    line4, = axs[2,0].plot(xxx, marg_vy, color='orange', ls='--', lw=2)
    plt.legend([line3, line4], ['$\\lambda_1$', '$\\lambda_2$' ])
    
    if normalize_marginals:
        maxht = 0.15
        vline_ht = 0.05 
    else:
        maxht = 350
        vline_ht = 100 # shows truth
        
    # slices through middle
    show_slice = False
    if show_slice:
        lines1, = axs[2,1].plot(xxx, zzs[int(num_plot_pts/2),:], '-', lw=3) # lam 1
        lines2, = axs[2,1].plot(xxx, zzs[:,int(num_plot_pts/2)], ':', lw=3) # lam 2
        linev1, = axs[2,0].plot(xxx, zzv[int(num_plot_pts/2),:], '-', lw=3) # lam 1
        linev2, = axs[2,0].plot(xxx, zzv[:,int(num_plot_pts/2)], ':', lw=3) # lam 2
        maxht = 10
    
    # VERTICAL LINE FOR TRUE VALUE
    axs[2,0].vlines(lam_true[0],0,vline_ht, 'k', lw=1)
    axs[2,0].vlines(lam_true[1],0,vline_ht, 'k', lw=1)
    axs[2,1].vlines(lam_true[0],0,vline_ht, 'k', lw=1)
    axs[2,1].vlines(lam_true[1],0,vline_ht, 'k', lw=1)
    axs[2,1].axis([-bound_val, bound_val, 0, maxht])
    axs[2,0].axis([-bound_val, bound_val, 0, maxht])
    plt.show()


In [68]:
colormaps = ['viridis', 'plasma', 'inferno', 'magma', 'jet', 
             'Greys', 'Purples', 'Blues', 'Greens', 'Oranges', 'Reds',
            'YlOrBr', 'YlOrRd', 'OrRd', 'PuRd', 'RdPu', 'BuPu',
            'GnBu', 'PuBu', 'YlGnBu', 'PuBuGn', 'BuGn', 'YlGn']

out = wd.interact(compare, 
            num_samples = wd.IntSlider(value=1000, min=100, max=5000, step=100, continuous_update=False), 
            skew = wd.FloatSlider(value=1.0, min=1.0, max=2.0, step=0.05, continuous_update=False), 
            prior_x = wd.FloatSlider(value=0.0, min=-0.25, max=0.25, step=0.05, continuous_update=False), 
            prior_y = wd.FloatSlider(value=0.0, min=-0.25, max=0.25, step=0.05, continuous_update=False),             
            prior_std = wd.FloatSlider(value=0.5, min=0.5, max=1.0, step=0.05, continuous_update=False), 
            data_std = wd.FloatSlider(value=0.25, min=0.05, max=0.5, step=0.01, continuous_update=False), 
            color_map = wd.Dropdown(value='jet', options=colormaps), 
            num_levels = wd.IntSlider(value=40, min=20, max=100, step=5, continuous_update=False),
            seed = wd.IntSlider(value=0, min=0, max=1000, step=1, continuous_update=False) )

interactive(children=(IntSlider(value=1000, continuous_update=False, description='num_samples', max=5000, min=…

# Observations

Bias prior away from observation and lower its variance. Make the data very uncertain. 
The MSE approach handles this well, while error starts to pollute the vector-valued approach.


In [19]:
out.widget.children[2].value = 1
out.widget.children[2].value = -0.5
out.widget.children[3].value = -0.5
out.widget.children[4].value = 0.75
out.widget.children[5].value = 0.25

